In [24]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR/'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2024-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-30,1,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-02-01,1,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2024-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81350,9.0,6.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,2.0,...,1.0,2.0,6.0,1.0,1.0,0.0,1.0,2024-11-26,265,1.0
81351,10.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,2.0,3.0,6.0,1.0,2024-11-27,265,0.0
81352,6.0,7.0,7.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,...,2.0,5.0,4.0,2.0,0.0,2.0,6.0,2024-11-28,265,0.0
81353,16.0,3.0,5.0,5.0,2.0,1.0,4.0,2.0,2.0,0.0,...,7.0,0.0,2.0,3.0,1.0,1.0,2.0,2024-11-29,265,2.0


In [25]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2024, 7, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40810, 674)
y_train.shape=(40810,)
X_test.shape=(40545, 674)
y_test.shape=(40545,)


In [26]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']

In [27]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
40540    1.0
40541    1.0
40542    6.0
40543    2.0
40544    1.0
Name: rides_previous_1_hour, Length: 40545, dtype: float32

In [28]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=7.2391


In [29]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_{7*24}_hour']

In [30]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [31]:
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.4264


In [32]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        """"""
        return 0.25*(
            X_test[f'rides_previous_{7*24}_hour'] + \
            X_test[f'rides_previous_{2*7*24}_hour'] + \
            X_test[f'rides_previous_{3*7*24}_hour'] + \
            X_test[f'rides_previous_{4*7*24}_hour']
        )

In [33]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.9194
